# Transfer learning

In [0]:
import tensorflow
import keras
import h5py

## Download the flower dataset

In [0]:
%sh
curl -O http://download.tensorflow.org/example_images/flower_photos.tgz
tar xzf flower_photos.tgz &>/dev/null
cd flower_photos
ls
pwd

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
 0 218M 0 495k 0 0 776k 0 0:04:47 --:--:-- 0:04:47 775k
 2 218M 2 4530k 0 0 2766k 0 0:01:20 0:00:01 0:01:19 2765k
 3 218M 3 8845k 0 0 3353k 0 0:01:06 0:00:02 0:01:04 3352k
 14 218M 14 31.7M 0 0 8929k 0 0:00:25 0:00:03 0:00:22 8929k
 36 218M 36 80.0M 0 0 16.8M 0 0:00:12 0:00:04 0:00:08 16.8M
 68 218M 68 149M 0 0 26.5M 0 0:00:08 0:00:05 0:00:03 29.7M
 92 218M 92 201M 0 0 30.3M 0 0:00:07 0:00:06 0:00:01 39.4M
100 218M 100 218M 0 0 31.1M 0 0:00:06 0:00:06 --:--:-- 48.0M
LICENSE.txt
daisy
dandelion
roses
sunflowers
tulips
/databricks/driver/flower_photos

In [0]:
img_dir = 'file:/databricks/driver/flower_photos'

## Load images into Dataframe

In [0]:
df = spark.read.format('image').load(img_dir)
df.show()
df.printSchema()

+--------------------+
 image|
+--------------------+
[file:/databricks...|
+--------------------+

root
-- image: struct (nullable = true)
 |-- origin: string (nullable = true)
 |-- height: integer (nullable = true)
 |-- width: integer (nullable = true)
 |-- nChannels: integer (nullable = true)
 |-- mode: integer (nullable = true)
 |-- data: binary (nullable = true)

## Create the train and test class

In [0]:
from pyspark.sql.functions import lit

In [0]:
# create Tulips and Sunflowers labels datasets, literally label each value as 0 (tulips) and 1 (sunflower) accordingly.

tulips_df = spark.read.format('image').load(img_dir + '/tulips').withColumn('label', lit(0))
sunflower_df = spark.read.format('image').load(img_dir + '/sunflowers').withColumn('label', lit(1))

In [0]:
# split train, test set
# tulips_train, tulips_test, _ = tulips_df.randomSplit([0.08, 0.08, 0.84])
# sunflower_train, sunflower_test, _ = sunflower_df.randomSplit([0.08, 0.08, 0.84])

tulips_train, tulips_test = tulips_df.randomSplit([0.8, 0.2], 42)
sunflower_train, sunflower_test = sunflower_df.randomSplit([0.8, 0.2], 42)

# combine those 2 flower types together again as whole train, test datasets
train_df = tulips_train.union(sunflower_train)
test_df = tulips_test.union(sunflower_test)

# as partition is quite expensive, we will re-partition to smaller block
train_df = train_df.repartition(100)
test_df = test_df.repartition(100)

## Create the custom classifier model, using pipeline

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from sparkdl import DeepImageFeaturizer

In [0]:
featurizer = DeepImageFeaturizer(inputCol='image', outputCol='features', modelName='ResNet50')

In [0]:
# we will use logistic regression because we want to classify 2 types of flowers
lr = LogisticRegression(maxIter=10, regParam=0.05, elasticNetParam=0.03, labelCol='label')

In [0]:
# we will use Pipeline
pipeline = Pipeline(stages=[featurizer, lr])

## Train the model

In [0]:
flower_model = pipeline.fit(train_df)

-----------

## Evaluate the Performance

In [0]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [0]:
pred = flower_model.transform(test_df)

In [0]:
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

print('Test dataset Accuracy: {}'.format(evaluator.evaluate(pred.select('prediction', 'label'))))

Test dataset Accuracy: 0.9647435897435898

## Using a test image

In [0]:
t = '/FileStore/tables/tulip.jpg'
s = '/FileStore/tables/sunflower.jpg'

In [0]:
# Load the images for testing

# tulips
df = spark.read.format('image').load(t)
df.show()

+--------------------+
 image|
+--------------------+
[dbfs:/FileStore/...|
+--------------------+

In [0]:
# make the predictions with our model
flower_model.transform(df).show()

+--------------------+--------------------+--------------------+--------------------+----------+
 image| features| rawPrediction| probability|prediction|
+--------------------+--------------------+--------------------+--------------------+----------+
[dbfs:/FileStore/...|[6.44039964675903...|[7.44890928007499...|[0.99941826246740...| 0.0|
+--------------------+--------------------+--------------------+--------------------+----------+

## model predictions result
our model has predicted that the image is 99.94% tulips, which is correct.

In [0]:
# sunflower image
df = spark.read.format('image').load(s)
df.show()

+--------------------+
 image|
+--------------------+
[dbfs:/FileStore/...|
+--------------------+

In [0]:
# make the predictions with our model
flower_model.transform(df).show()

+--------------------+--------------------+--------------------+--------------------+----------+
 image| features| rawPrediction| probability|prediction|
+--------------------+--------------------+--------------------+--------------------+----------+
[dbfs:/FileStore/...|[0.83528298139572...|[-6.7756860355786...|[0.00113988651217...| 1.0|
+--------------------+--------------------+--------------------+--------------------+----------+

## model predictions result
This time our model predicted that image is sunflower with probability of 0.113%, which is not very confident. but it correctly classfied the flower type though.